In [ ]:
import requests
import json
import base64
import numpy as np
from PIL import Image

# 이미지를 Base64로 인코딩하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as img:
        return base64.b64encode(img.read()).decode("utf-8")

# PNG 이미지를 JPG로 변환하는 함수
def convert_to_jpg(image_path):
    img = Image.open(image_path)
    rgb_im = img.convert("RGB")  # RGB로 변환하여 색상 손실 방지
    new_path = image_path.replace(".png", ".jpg")  # 파일 확장자 변경
    rgb_im.save(new_path, "JPEG")  # JPEG 포맷으로 저장
    return new_path

# 이미지를 특정 크기로 리사이징하는 함수
def resize_image(image_path, max_size=(224, 224)):
    img = Image.open(image_path)
    img.thumbnail(max_size)  # 최대 크기로 조정
    new_path = image_path.replace(".jpg", "_resized.jpg")  # 파일명 변경
    img.save(new_path, "JPEG")  # JPEG 포맷으로 저장
    return new_path

# API 요청을 보내는 함수
def send_request(api_url, api_key, data):
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
    response = requests.post(api_url, headers=headers, json=data)
    try:
        return response.json()  # JSON 응답 반환
    except json.JSONDecodeError:
        return None  # JSON 파싱 오류 시 None 반환

# 이미지 파일 경로 설정
image_file = r"IMAGE PATH"
image_file = convert_to_jpg(image_file)  # PNG를 JPG로 변환
image_file = resize_image(image_file)  # 이미지 크기 조정
image_base64 = encode_image(image_file)  # Base64로 변환

# 1차 분류 (과일/채소 구분)
api_url_1 = "YOUR API URL"
api_key_1 = "YOUR API KEY"

data_1 = {
    "Inputs": {"WebServiceInput0": [{"image": image_base64, "id": 0, "category": "Fruit"}]},
    "GlobalParameters": {}
}

response_data_1 = send_request(api_url_1, api_key_1, data_1)
if not response_data_1:
    exit()

result_1 = response_data_1.get("Results", {}).get("WebServiceOutput0", [])[0]
label_1 = result_1.get("Scored Labels", "Unknown")
print(f"Step 1 - Category: {label_1}")

# 2차 분류 (신선도 판별)
if label_1 == "Fruit":
    api_url_2 = "YOUR API URL"
    api_key_2 = "YOUR API KEY"
else:
    api_url_2 = "YOUR API URL"
    api_key_2 = "YOUR API KEY"

data_2 = {
    "Inputs": {"WebServiceInput0": [{"image": image_base64, "id": 0, "category": "Fresh"}]},
    "GlobalParameters": {}
}

response_data_2 = send_request(api_url_2, api_key_2, data_2)
if not response_data_2:
    exit()

result_2 = response_data_2.get("Results", {}).get("WebServiceOutput0", [])
if result_2:
    best_result = max(result_2, key=lambda x: x.get(f"Scored Probabilities_{x['Scored Labels']}", 0))
    label_2 = best_result.get("Scored Labels", "Unknown")
    probability_2 = best_result.get(f"Scored Probabilities_{label_2}", 0) * 100
    print(f"Step 2 - Fresh/Rotten: {label_2} ({probability_2:.2f}%)")
else:
    exit()

# 3차 분류 (세부 종류 판별)
if label_1 == "Fruit":
    api_url_3 = "YOUR API URL"
    api_key_3 = "YOUR API KEY"
    categories = ["Apple", "Banana", "Kiwi", "Mango", "Orange", "Pineapple", "Strawberry", "Watermelon"]
else:
    api_url_3 = "YOUR API URL"
    api_key_3 = "YOUR API KEY"
    categories = ["Cabbage", "Carrot", "Cucumber", "Eggplant", "Garlic", "Onion", "Pepper", "Tomato"]

data_3 = {
    "Inputs": {"WebServiceInput0": [{"image": image_base64, "id": i, "category": category} for i, category in enumerate(categories)]},
    "GlobalParameters": {}
}

response_data_3 = send_request(api_url_3, api_key_3, data_3)
if not response_data_3:
    exit()

result_3 = response_data_3.get("Results", {}).get("WebServiceOutput0", [])
if result_3:
    best_result_3 = max(result_3, key=lambda x: x.get(f"Scored Probabilities_{x['Scored Labels']}", 0))
    label_3 = best_result_3.get("Scored Labels", "Unknown")
    probability_3 = best_result_3.get(f"Scored Probabilities_{label_3}", 0) * 100
    print(f"Step 3 - Final Category: {label_3} ({probability_3:.2f}%)")